# Notebook compilation of data
An attempt to compile known GC data, and select the best/latest available.

## TODO
- [ ] Make sensible selection of best
- [ ] Check Gran Physical Coords
- [ ] Check Systematics of AMR  
    - [ ] Differences in Isochrones, BaSTi, DSED, BaSTi Alpha enhanced, BaSTI sometimes corrected BaSTi with an age shift due to atomic diffusion ect
    

## Name Preferencse
Alt Names gives preferences of  variants on a single name  
name_parse gives the preferences of alternative names of a GC


In [1]:
import numpy as np

In [2]:
from ParseNames import process_names, name_parse,alt_name_parse, create_alt_name_list
from ParseProperty import property_parser, process_properties

In [3]:
from DataFuncs import data_add_function, data_dic_to_gc_dic, data_array_to_gc_dic, gc_dic_to_data_dic

In [4]:
paper_age = {}
data = {}

# Larger Data Files
Read in csv/txt files
## Dynamics

### Vasiliev

In [5]:
def load_Vasilev21_Data_Dic():
    # Eugene Vasilev read script
    tab = np.loadtxt('DataFiles/GC_Sky_Vasilev2021.txt', dtype=str)
    print(np.shape(tab))
    sky_data = {}
    sky_data['name'] = tab[:, 0]   # 0th column is the cluster name (string)
    sky_data['name2'] = tab[:, 1]   # 0th column is the cluster name (string)
    tab = tab[:, 2:].astype(float)  # remaining columns are numbers
    sky_data['ra_deg'] = tab[:, 0]   # coordinates of cluster centers [deg]
    sky_data['dec_deg'] = tab[:, 1]
    sky_data['dist'] = tab[:, 2]   # distance [kpc]
    diste = tab[:, 3]   # distance error [kpc]
    diste[~np.isfinite(diste)] = sky_data['dist'][~np.isfinite(diste)] * 0.05
    sky_data['dist_err'] = diste   # distance error [kpc]
    sky_data['vlos'] = tab[:, 4]   # line-of-sight velocity [km/s]
    vlose = tab[:, 5]   # its error estimate
    # assumed error of at least 2 km/s on line-of-sight velocity
    sky_data['vlos_err'] = np.maximum(vlose, 2.0)
    sky_data['pmra'] = tab[:, 8]   # mean proper motion [mas/yr]
    sky_data['pmdec'] = tab[:, 9]
    sky_data['pmra_err'] = tab[:, 10]   # its uncertainty
    sky_data['pmdec_err'] = tab[:, 11]
    # correlation coefficient for errors in two PM components
    sky_data['pmcorr'] = tab[:, 12]
    # assumed error of 0.1 mag in distance modulus when not known
    return sky_data


Vasilev21_data_dic = load_Vasilev21_Data_Dic()
Vasilev21_dic = data_dic_to_gc_dic(Vasilev21_data_dic)
data = data_add_function(data, Vasilev21_dic, data_name='Vasilev21')
paper_age["Vasilev21"] = 2021

(170, 18)


### Baumb Distances

In [6]:
def Baumgardt_GC_obs_load_sky():
    tab = np.loadtxt("DataFiles/Baumgardt21_sky.txt", dtype=str, skiprows=2)
    sky_data = {}
    Name = tab[:, 0].astype('<U20')   # 0th column is the cluster name (string)
    Name = np.char.replace(Name, ' ', '')
    Name = np.char.replace(Name, '_', '')
    Name = np.char.replace(Name, '-', '')
    Name = np.char.replace(Name, 'Ter', 'Terzan')
    Name = np.char.replace(Name, 'Lae', 'Laevens')
    Name = np.char.replace(Name, 'Djor', 'Djorg')
    Name = np.char.replace(Name, 'Lil', 'Liller')
    sky_data['name'] = Name
    tab = tab[:, 1:].astype(float)  # remaining columns are numbers
    sky_data['ra_deg'] = tab[:, 0]   # coordinates of cluster centers [deg]
    sky_data['dec_deg'] = tab[:, 1]
    sky_data['l_deg'] = tab[:, 2]   # coordinates of cluster centers [deg]
    sky_data['b_deg'] = tab[:, 3]   # coordinates of cluster centers [deg]
    sky_data['dist'] = tab[:, 4]   # distance [kpc]
    diste = tab[:, 5]   # distance error [kpc]
    diste[~np.isfinite(diste)] = sky_data['dist'][~np.isfinite(diste)] * 0.05
    sky_data['dist_err'] = diste   # distance error [kpc]
    # sky_data['R_GC'] = tab[:, 6]
    sky_data['vlos'] = tab[:, 7]   # line-of-sight velocity [km/s]
    vlose = tab[:, 8]   # its error estimate
    # np.maximum(vlose, 2.0)  # assumed error of at least 2 km/s on line-of-sight velocity
    sky_data['vlos_err'] = vlose
    sky_data['pmra'] = tab[:, 9]   # mean proper motion [mas/yr]
    sky_data['pmra_err'] = tab[:, 10]   # its uncertainty
    sky_data['pmdec'] = tab[:, 11]
    sky_data['pmdec_err'] = tab[:, 12]
    # correlation coefficient for errors in two PM components
    sky_data['pmcorr'] = tab[:, 13]
    # assumed error of 0.1 mag in distance modulus when not known
    return sky_data


Baumgardt21_data_dic = Baumgardt_GC_obs_load_sky()
Baumgardt21_dic = data_dic_to_gc_dic(Baumgardt21_data_dic)
data = data_add_function(data, Baumgardt21_dic, data_name='Baumgardt21')
paper_age["Baumgardt21"] = 2021.5

New GC: GLIMPSE01
New GC: GLIMPSE02
New GC: SagittariusII


## AMR + Mass/Magnitude

### Baumgardt Cluster Mass
https://ui.adsabs.harvard.edu/abs/2018MNRAS.478.1520B/abstract  
https://people.smp.uq.edu.au/HolgerBaumgardt/globular/parameter.html
Fits N-body Simulatons to velocity dispersion and surface density profiles.

In [7]:
Baumgardt_GC_array = np.loadtxt("DataFiles/Baumgardt_GC_Mass.csv",
                                skiprows=1, dtype=str, delimiter=',')
Baumgardt_GC_Columns = ["Name", "Mass"]
print(np.shape(Baumgardt_GC_array))
Baumgardt_GC_dic = data_array_to_gc_dic(Baumgardt_GC_array, Baumgardt_GC_Columns)
data = data_add_function(data, Baumgardt_GC_dic, data_name='Baumgardt_Mass18')
paper_age["Baumgardt_Mass18"] = 2018

(161, 2)
New GC: 2MASS
New GC: 2MASS


### Kruijssen19 Sample
https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.3180K/abstract
Data used in Kruijssen19. 3 Sources, Kuijssen takes average of available. Use all from sources, as some missing data added by other sources.

In [8]:
Kruijssen19_array = np.loadtxt("DataFiles/Kruijssen19.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Met, Alpha, Age, Age Err
Kruijssen19_Columns = ['Name', 'Mv', 'Mass', 'Age', 'Age_Err', 'Fe_H']
Kruijssen19_array[:, 2] = (10 ** np.array(Kruijssen19_array[:, 2].astype(float))).astype(str)
Kruijssen19_dic = data_array_to_gc_dic(Kruijssen19_array, Kruijssen19_Columns)
data = data_add_function(data, Kruijssen19_dic, data_name='Kruijssen19')
paper_age["Kruijssen19"] = 2013

#### Dotter
Dotter 11 added onto the back of the datafile. 
https://ui.adsabs.harvard.edu/abs/2010ApJ...708..698D/abstract  
https://ui.adsabs.harvard.edu/abs/2011ApJ...738...74D/abstract  

Fit Dartmouth isochrones to CMD data.  
Dartmouth Isochrones (DSEP) used detailed in https://ui.adsabs.harvard.edu/abs/2007AJ....134..376D/abstract   
Not a commonly used isochrone - biases?
DSEP not good add low metallicity?

Best fit age from isochrone MSTO through the subgiant branch.


In [9]:
Dotter10_array = np.loadtxt("DataFiles/Dotter10.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Met, Alpha, Age, Age Err
Dotter10_Columns = ['Name', 'Fe_H', 'Alpha_Fe', 'Age', 'Age_Err']
print(np.shape(Dotter10_array))
Dotter10_dic = data_array_to_gc_dic(Dotter10_array, Dotter10_Columns)
data = data_add_function(data, Dotter10_dic, data_name='Dotter10')
paper_age['Dotter10'] = 2010

(62, 5)
New GC: Ruprecht106


#### Forbes+Bridges10
https://ui.adsabs.harvard.edu/abs/2010MNRAS.404.1203F/abstract   
Note No Age errors?  

Already a compilation of data!
- 64 from  Marin-Franch 2009  (Dartmouth Isochrone), Normalised by assuming absolute metal poor age?
- 13 from Angeli 2005, assuming same norm
- 10 from Salaris 1995.
- + Others
Most comes from Marin-Franch

In [10]:
Forbes10_array = np.loadtxt("DataFiles/Forbes10.csv", skiprows=1, dtype=str, delimiter=',')
Forbes10_Columns = ['Name', 'Age', 'Met']
# No Age error?
print(np.shape(Forbes10_array))
Forbes10_dic = data_array_to_gc_dic(Forbes10_array, Forbes10_Columns)
data = data_add_function(data, Forbes10_dic, data_name='Forbes10')
paper_age["Forbes10"] = 2010

(88, 3)
New GC: Berk29
New GC: Saurer1


#### VandenBerg13
https://ui.adsabs.harvard.edu/abs/2013ApJ...775..134V/abstract  
Following Kruijssen, +1Gyr Uncertainty:  
''' For the VandenBerg et al. (2013) sample, the uncertainties include the statistical uncertainty listed in their paper, as well as an additional uncertainty of 1 Gyr to account for uncertainties in distance and chemical abundances'''  
Isochrone fit to MSTO

In [11]:
VandenBerg13_array = np.loadtxt("DataFiles/VandenBerg13.csv", skiprows=1, dtype=str, delimiter=',')
#NGC, Name, Met, Age, Age_Err, Mv
VandenBerg13_Columns = ['NGC', 'Name', 'Met', 'Age', 'Age_Err', 'Mv']
print(np.shape(VandenBerg13_array))
print("Adding 1GYR to errors")
VandenBerg13_array[:, 4] = (VandenBerg13_array[:, 4].astype(float) + 1).astype(str)
VandenBerg13_dic = data_array_to_gc_dic(VandenBerg13_array, VandenBerg13_Columns)
data = data_add_function(data, VandenBerg13_dic, data_name='VandenBerg13')
paper_age["VandenBerg13"] = 2013

(55, 6)
Adding 1GYR to errors


### Harris10
https://physics.mcmaster.ca/~harris/mwgc.dat

In [12]:
Harris10_array = np.loadtxt("DataFiles/Harris10.csv", skiprows=1, dtype=str, delimiter=',')
#Name,Met, Mvt
Harris10_Columns = ['Name', 'Met', 'Mvt']
Harris10_dic = data_array_to_gc_dic(Harris10_array, Harris10_Columns)
data = data_add_function(data, Harris10_dic, data_name='Harris10')

paper_age["Harris10"] = 2010

### Roediger13
https://ui.adsabs.harvard.edu/abs/2014ApJS..210...10R/abstract

In [13]:
Roediger13_array = np.loadtxt("DataFiles/Roediger13.csv", skiprows=1, dtype=str, delimiter=',')
Roediger13_Columns = ['Name', 'Age', 'Age_Err', 'Fe_H', 'Fe_H_err']
print(np.shape(Roediger13_array))
Roediger13_dic = data_array_to_gc_dic(Roediger13_array, Roediger13_Columns)
data = data_add_function(data, Roediger13_dic, data_name='Roediger13')
paper_age["Roediger13"] = 2013

(41, 5)


### OMalley17
https://ui.adsabs.harvard.edu/abs/2017ApJ...838..162O/abstract

In [14]:
OMalley17_array = np.loadtxt("DataFiles/OMalley17.csv", skiprows=1, dtype=str, delimiter=',')
#Name, m_Mv, Age, Age_Err, Fe_H
OMalley17_Columns = ["Name", "m_Mv", "m_Mv_Err", "Age", "Age_Err", "Fe_H"]
print(np.shape(OMalley17_array))
OMalley17_dic = data_array_to_gc_dic(OMalley17_array, OMalley17_Columns)
data = data_add_function(data, OMalley17_dic, data_name='OMalley17')

paper_age["OMalley17"] = 2017

(22, 6)


### Vasquez18
https://ui.adsabs.harvard.edu/abs/2018A%26A...619A..13V/abstract  
Use  the most recent Fe_H_D16 data?  
Also contains radial velocities!

In [15]:
Vasquez18_array = np.loadtxt("DataFiles/Vasquez18.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Fe_H D16, Fe_H_err, Fe_H_S12, Fe_H_err, Fe_H_V15, Fe_H_err, Fe_H_C09
Vasquez18_array = Vasquez18_array[:, :3]
Vasquez18_Columns = ["Name", "Fe_H", "Fe_H_err"]
print(np.shape(Vasquez18_array))
Vasquez18_dic = data_array_to_gc_dic(Vasquez18_array, Vasquez18_Columns)
data = data_add_function(data, Vasquez18_dic, data_name='Vasquez18')

paper_age["Vasquez18"] = 2018

(48, 3)


### Saviane12
https://ui.adsabs.harvard.edu/abs/2012A%26A...540A..27S/abstract

Use Fe_H_C09 data?

In [16]:
Saviane12_array = np.loadtxt("DataFiles/Saviane12.csv", skiprows=1, dtype=str, delimiter=',')
#Name, Name2, W, sigmaW, Fe_H_C09, err, Fe_H_H10, H10wt, Fe_H_C09_AP1, e
Saviane12_array = Saviane12_array[:, [0, 1, 4, 5]]
print(np.shape(Saviane12_array))
Saviane12_Columns = ["Name", "Name2", "Fe_H", "Fe_H_err"]
Saviane12_dic = data_array_to_gc_dic(Saviane12_array, Saviane12_Columns)
data = data_add_function(data, Saviane12_dic, data_name='Saviane12')

paper_age["Saviane12"] = 2012

(71, 4)


### Oliveria20
https://ui.adsabs.harvard.edu/abs/2020ApJ...891...37O/abstract  
HST


In [17]:
Oliveria20_array = np.loadtxt("DataFiles/Oliveria20.csv", skiprows=1, dtype=str, delimiter=',')
Oliveria20_Columns = ["Name", "Age", "Age_Err_1os",
                      "Age_Err_1us", "Fe_H", "Fe_H_Err_1os", "Fe_H_Err_1us"]
Oliveria20_array[:, 2] = (10 ** np.array(Oliveria20_array[:, 2].astype(float))).astype(str)
Oliveria20_dic = data_array_to_gc_dic(Oliveria20_array, Oliveria20_Columns)
data = data_add_function(data, Oliveria20_dic, data_name='Oliveria20')
paper_age["Oliveria20"] = 2020

### CAPOS21
https://ui.adsabs.harvard.edu/abs/2021A%26A...652A.157G/abstract 
Further Chemistry Available! 


In [18]:
CAPOS21_array = np.loadtxt("DataFiles/CAPOS21.csv", skiprows=1, dtype=str, delimiter=',')
CAPOS21_Columns = ["Name", "Fe_H", "Fe_H_Err", "Alpha_Fe", "Alpha_Fe_Err"]
CAPOS21_array[:, 2] = (10 ** np.array(CAPOS21_array[:, 2].astype(float))).astype(str)
CAPOS21_dic = data_array_to_gc_dic(CAPOS21_array, CAPOS21_Columns)
data = data_add_function(data, CAPOS21_dic, data_name='CAPOS21')
paper_age["CAPOS21"] = 2020

### GRAN22
Gran22: https://ui.adsabs.harvard.edu/abs/2022MNRAS.509.4962G/abstract  
Errors?  
Correct form of PMs?

In [19]:

Gran22_array = np.loadtxt("DataFiles/Gran22.csv", skiprows=1, dtype=str, delimiter=',')
Gran22_Columns = ["Name", "l_deg", "b_deg", "ra_deg", "dec_deg", "mua_cos", "mud",
                  "mul_cosb", "mub", "Fe_H", "Fe_H_Err", "Mass", "Mass_Err", "Dist", "Mv", "vlos", "vlos_Err"]

Gran22_array[:, 2] = (10 ** np.array(Gran22_array[:, 2].astype(float))).astype(str)
Gran22_dic = data_array_to_gc_dic(Gran22_array, Gran22_Columns)
data = data_add_function(data, Gran22_dic, data_name='Gran22')
paper_age["Gran22"] = 2020

New Prop: mua_cos
New Prop: mud
New Prop: mul_cosb
New Prop: mub
New GC: Gran2
New GC: Gran3
New GC: Gran4
New GC: Gran5


### Valenti09
Colour magnitude  
Paper: https://ui.adsabs.harvard.edu/abs/2010MNRAS.402.1729V/abstract  
Data: http://www.bo.astro.it/~GC/ir_archive/  
Compilation of Previous Valenti data - should be updated reguarly?

In [20]:
Valenti09_array = np.loadtxt("DataFiles/Valenti09.csv", skiprows=1, dtype=str, delimiter=',')
original_Valenti09_Columns = ["Name", "l_deg", "b_deg", "dsun", "RGC", "X", "Y", "Z", "E(B-V)", "(m-M)0", "Fe_H",
                     "[M/H]", "JRC", "HRC", "KRC", "JBump", "HBump", "KBump", "JTip", "HTip", "KTip",
                     "MBolBump", "MBolTip"]

Valenti09_Columns = ["Name", "l_deg", "b_deg", "dsun", "RGC", "Fe_H"]
Valenti09_filt = np.isin(np.array(original_Valenti09_Columns), np.array(Valenti09_Columns))
print(np.shape(Valenti09_array))
Valenti09_array = Valenti09_array[:,Valenti09_filt]
print(np.shape(Valenti09_array))


Valenti09_dic = data_array_to_gc_dic(Valenti09_array, Valenti09_Columns)
data = data_add_function(data, Valenti09_dic, data_name='Valenti09')
paper_age["Valenti09"] = 2009

(36, 23)
(36, 6)
New Prop: RGC


### Bica16
https://ui.adsabs.harvard.edu/abs/2016PASA...33...28B/abstract
Candidate GCs with sky, Metallicity and further Chem data.  
Not clear which are actually GCs or not


# Individual_Data
Individual papers and sources

In [21]:
def add_individual_data(data, individual_data):
    papers = list(individual_data.keys())
    for p in papers:
        print(p)
        original_new_data = individual_data[p]
        original_names = np.array(list(original_new_data.keys()))
        names = process_names(original_names)
        try:
            new_data = {new_n: original_new_data[old_n] for (new_n, old_n) in zip(names, original_names)}
            data = data_add_function(data, new_data, data_name=p)
        except Exception as e:
            print(e)
            print(names)
            print(original_names)
            print(original_new_data.keys())

    return data

In [22]:
individual_data = {}

## Small Papers

### Kerber18
NGC6522, NGC6626(M28), NGC6362  
Kerber18: https://ui.adsabs.harvard.edu/abs/2018ApJ...853...15K/abstract  
Alpha enhanced BASTI or DSED sochrone fitting. Use DSED to avoid attomic difusion problems.  
Take the smallest ChiSq.

In [23]:
individual_data["Kerber18"] = {
    "NGC6522": {
        "Age": 11.4, "Age_Err": 1.0,
        "Fe_H": -1.15,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 7.05, "dist_err": 0.16
    },
    "NGC6626": {
        "Age": 11.1, "Age_Err": 0.9,
        "Fe_H": -1.3,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 5.18, "dist_err": 0.14
    },
    "NGC6362": {
        "Age": 12.8, "Age_Err": 1.0,
        "Fe_H": -1.15,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 7.73, "dist_err": 0.18
    }
}

paper_age["Kerber18"] = 2018

### Dias15
NGC6528, NGC6553, M71, NGC6558, NGC6426, Terzan8
Dias15: https://ui.adsabs.harvard.edu/abs/2015A%26A...573A..13D/abstract  
Fe_H, Alpha_Fe, Mg, vlos

In [24]:
individual_data["Dias15"] = {
    "NGC6528": {
        "Fe_H": -0.13, "Fe_H_Err": 0.05,
        "vlos": 185, "vlos_err": 10,
        "Mg_Fe": 0.05, "Mg_Fe_Err": 0.09,
        "Alpha_Fe": 0.26, "Alpha_Fe_Err": 0.05,
    },
    "NGC6553": {
        "Fe_H": -0.133, "Fe_H_Err": 0.017,
        "vlos": 6, "vlos_err": 8,
        "Mg_Fe": 0.107, "Mg_Fe_Err": 0.009,
        "Alpha_Fe": 0.302, "Alpha_Fe_Err": 0.025,
    },
    "M71": {
        "Fe_H": -0.63, "Fe_H_Err": 0.05,
        "vlos": -42, "vlos_err": 18,
        "Mg_Fe": 0.25, "Mg_Fe_Err": 0.07,
        "Alpha_Fe": 0.293, "Alpha_Fe_Err": 0.032,
    },
    "NGC6558": {
        "Fe_H": -1.012, "Fe_H_Err": 0.013,
        "vlos": -210, "vlos_err": 16,
        "Mg_Fe": 0.26, "Mg_Fe_Err": 0.06,
        "Alpha_Fe": 0.23, "Alpha_Fe_Err": 0.06,
    },
    "NGC6426": {
        "Fe_H": -2.39, "Fe_H_Err": 0.11,
        "vlos": -242, "vlos_err": 11,
        "Mg_Fe": 0.38, "Mg_Fe_Err": 0.06,
        "Alpha_Fe": 0.24, "Alpha_Fe_Err": 0.05,
    },
    "Terzan8": {
        "Fe_H": -2.06, "Fe_H_Err": 0.17,
        "vlos": 135, "vlos_err": 19,
        "Mg_Fe": 0.41, "Mg_Fe_Err": 0.04,
        "Alpha_Fe": 0.21, "Alpha_Fe_Err": 0.04,
    }
}

paper_age["Dias15"] = 2015

### VandenBerg16
https://ui.adsabs.harvard.edu/abs/2016ApJ...827....2V/abstract  
M3, M15, M92. More information, but not very clear?

In [25]:
individual_data["Vandenberg16"] = {
    "M3": {
        "Age": 12.5
    },
    "M15": {
        "Age": 12.5
    },
    "M92": {
        "Age": 12.5
    },
}

paper_age["Vandenberg16"] = 2016

## Individual Clusters

### Pal2
https://ui.adsabs.harvard.edu/abs/2020MNRAS.493.2688B/abstract

In [26]:
individual_data["Bonatto20"] = {"Pal2": {'Age': 13.25, 'Age_Err': 0.12, "Fe_H": -1.58, "Fe_H_Err": 0.08,
                                         "Mass": 1.4e5, "Mass_Err": 0.4e5, "Mvt": -7.8}}
paper_age["Bonatto20"] = 2020

### Crater/Lavens1
Crater  
https://ui.adsabs.harvard.edu/abs/2016ApJ...822...32W/abstract

In [27]:

individual_data["Weisz16"] = {"Crater": {'Age': 7.5, 'Age_Err': 0.4,
                                         "Fe_H": -1.66, "Fe_H_Err": 0.04,
                                         "Mass": 9.9e3, "Mvt": -5.3}}
paper_age["Weisz16"] = 2016

### Bliss1
https://ui.adsabs.harvard.edu/abs/2019ApJ...875..154M/abstract

In [28]:
individual_data["Hempel14"] = {"Bliss1": {'Age': 9.2, 'Age_Err_1os': 1.6, "Age_Err_1us": -0.8,
                                          "Fe_H": -1.4,
                                          "Mass": 143, "Mass_Err": 37,
                                          "Mvt": 0, "Mvt_Err_1os": 1.7, "Mvt_Err_1us": -0.7}}
paper_age["Hempel14"] = 2014

### Kim3
https://ui.adsabs.harvard.edu/abs/2016ApJ...820..119K/abstract

In [29]:
individual_data["Kim16"] = {"Kim3": {'Age': 9.5, 'Age_Err_1os': 3, "Age_Err_1us": -1.6,
                                     "Fe_H": -1.6, "Fe_H_Err_1os": 0.45, "Fe_H_Err_1us": -0.3,
                                     "Mvt": 0.7, "Mvt_Err": 0.3}}
paper_age["Kim16"] = 2016

### Munoz1
https://ui.adsabs.harvard.edu/abs/2012ApJ...753L..15M/abstract

In [30]:
individual_data["Munoz12"] = {"Munoz1": {'Age': 12.5,
                                         "Fe_H": -1.5,
                                         "Mvt": -0.4, "Mvt_Err": 0.9}}

paper_age["Munoz12"] = 2012

### BH176
https://ui.adsabs.harvard.edu/abs/2011A%26A...528A..70D/abstract

In [31]:
individual_data["Davoust11"] = {"BH176": {'Age': 6.5, "Age_Err": 0.5,
                                          "Fe_H": -0.1, "Fe_H_Err": 0.1,
                                          "Mvt": -3.82}}

paper_age["Davoust11"] = 2011

### FSR1716
https://ui.adsabs.harvard.edu/abs/2017A%26A...605A.128K/abstract

Age given as 10-12 in:
https://ui.adsabs.harvard.edu/abs/2016arXiv161103753B/abstract

In [32]:
individual_data["Andreas17"] = {"FSR1716": {  # 'Age': ?,
    "Fe_H": -1.38, "Fe_H_Err": 0.2,
    "Mass": 1.4e4, "Mass_Err_1os": 1.2e4, "Mass_Err_1us": 0.8e4,
    "Mvt": -5.1, "Mvt_Err": 1}}

paper_age["Andreas17"] = 2017

individual_data["Buckner16"] = {"FSR1716": {'Age': 11, "Age_Err": 1}}
paper_age["Buckner16"] = 2016

### Ryu 1 and 2
https://ui.adsabs.harvard.edu/abs/2018ApJ...863L..38R/abstract  
Age assumed 12.6, as an average of metal poor GCs. Could include it and have very wide errors?

In [33]:
individual_data["Ryu18"] = {
    "Ryu059": {  # 'Age': 12.6,
        "Fe_H": -2.2, "Fe_H_Err": 0.2, },
    "Ryu879": {  # 'Age': 12.6,
        "Fe_H": -2.1, "Fe_H_Err": 0.3, }
}

paper_age["Ryu18"] = 2018

### NGC6139
https://ui.adsabs.harvard.edu/abs/2015A%26A...583A..69B/abstract
errors of +- 0.015 +- 0.058  
Combine in Quadrature

In [34]:
individual_data["Bragaglia15"] = {"NGC6139": {
    "Fe_H": -1.579, "Fe_H_Err": np.sqrt((0.015**2) + (0.058**2)),
    "Mvt": -3.82}}

paper_age["Bragaglia15"] = 2015

### ESO452
Simpson 17
https://ui.adsabs.harvard.edu/abs/2017MNRAS.472.2856S/abstract


Cornish 06. 
Fits multiple isochrones, find a large range of metallicity and ages.
https://ui.adsabs.harvard.edu/abs/2006AJ....131.2543C/abstract

In [35]:
individual_data["Simpson17"] = {"ESO452": {
    "Fe_H": -0.81, "Fe_H_Err": 0.13,
    "Mass": 6.8e3, "Mass_Err": 3.4e3
}
}

paper_age["Simpson17"] = 2017

individual_data["Cornish06"] = {"ESO452": {"Age": 12.5, "Age_Err": 3.5,
                                           }
                                }
paper_age["Cornish06"] = 2006

### NGC6229
Johnson17: https://ui.adsabs.harvard.edu/abs/2017AJ....154..155J/abstract

Arlleno15: https://ui.adsabs.harvard.edu/abs/2015MNRAS.452..727A/abstract  
Gives age as consistent with 12+-1


In [36]:
individual_data["Johnson17"] = {"NGC6229": {
    "Fe_H": -1.13, "Fe_H_Err": 0.06,
}}

paper_age["Johnson17"] = 2017

individual_data["Arellano15"] = {"NGC6229": {"Age": 12, "Age_Err": 1,
                                             "Fe_H": -1.31, "Fe_H_Err": 0.12,
                                             }}
paper_age["Arellano15"] = 2015

### NGC6256
Cardelano20: https://ui.adsabs.harvard.edu/abs/2020ApJ...895...54C/abstract  
Isochrone fitting using Vasq18 Metallicity


In [37]:
individual_data["Cardelano20"] = {"NGC6256": {
    "Age": 13, "Age_Err": 0.5,
    "Fe_H": -1.62
}}

paper_age["Cardelano20"] = 2020

### HP1
Kerber20: https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.5530K/abstract

Barbuy16: https://ui.adsabs.harvard.edu/abs/2016A%26A...591A..53B/abstract

In [38]:
individual_data["Kerber20"] = {"HP1": {
    "Age": 12.75, "Age_Err_1os": 0.86, "Age_Err_1us": -0.81,
    "Fe_H": -1.09, "Fe_H_Err_1os": 0.07, "Fe_H_Err_1us": -0.09,
    "Alpha_Fe": 0.4
}}

paper_age["Kerber20"] = 2020

individual_data["Barbuy16"] = {"HP1": {
    "Fe_H": -1.06, "Fe_H_Err": 0.1,
}}

paper_age["Barbuy16"] = 2016

### FSR1758
Romero21: https://ui.adsabs.harvard.edu/abs/2021A%26A...652A.158R/abstract  
Fits Isochrone, CAPOS APOGEE 

Vilanova19: https://ui.adsabs.harvard.edu/abs/2019ApJ...882..174V/abstract  
High Res Spectra. Prefered?

Barba19: https://ui.adsabs.harvard.edu/abs/2019ApJ...870L..24B/abstract

In [39]:
individual_data["Romero21"] = {"FSR1758": {
    "Age": 11.6, "Age_Err_1os": 1.25, "Age_Err_1us": -1.31,
    "Fe_H": -1.36, "Fe_H_Err": 0.05
}}

paper_age["Romero21"] = 2021

individual_data["Vilanova19"] = {"FSR1758": {
    "Fe_H": -1.58, "Fe_H_Err": 0.03,
    "Alpha_Fe": 0.32, "Alpha_Fe_Err": 0.01
}}

paper_age["Vilanova19"] = 2019

individual_data["Barba19"] = {"FSR1758": {
    "Fe_H": -1.5, "Fe_H": 0.3
}}

paper_age["Barba19"] = 2019

### NGC6402
DAntona22: https://ui.adsabs.harvard.edu/abs/2022ApJ...925..192D/abstract  
Johnson19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.485.4311J/abstract

In [40]:
individual_data["DAntona22"] = {"NGC6402": {
    "Age": 12.5, "Age_Err": 0.75,
}}

paper_age["DAntona22"] = 2022

individual_data["Johnson19"] = {"NGC6402": {
    "Fe_H": -1.13, "Fe_H_Err": 0.05,
    "Alpha_Fe": 0.3
}}

paper_age["Johnson19"] = 2019

### Palomar 6
Souza21: https://ui.adsabs.harvard.edu/abs/2021A%26A...656A..78S/abstract

In [41]:
individual_data["Souza21"] = {"Pal6": {
    "Age": 12.4, "Age_Err": 0.9,
    "Fe_H": -1.10, "Fe_H_Err": 0.09,
    "Mvt": -6.79
}}

paper_age["Souza21"] = 2021

### NGC6440
Pallanca21: https://ui.adsabs.harvard.edu/abs/2021ApJ...913..137P/abstract

Munoz17: https://ui.adsabs.harvard.edu/abs/2017A%26A...605A..12M/abstract  
Na-0 anticorrelation  

In [42]:
individual_data["Pallanca21"] = {"NGC6440": {
    "Age": 13, "Age_Err": 1.5,
    "Fe_H": -1.34, "Fe_H_Err": 0.36,
    "Mvt": 1.12, "Mvt_Err": 0.12
}}

paper_age["Pallanca21"] = 2021

individual_data["Munoz17"] = {"NGC6440": {
    "Fe_H": -0.5, "Fe_H_Err": 0.03,
    "Age": 13
}}

paper_age["Munoz17"] = 2017

### UKS1
Fernandez20: https://ui.adsabs.harvard.edu/abs/2020A%26A...643A.145F/abstract  
Spectra of 6 stars, fit isochrone for age

In [43]:
individual_data["Fernandez20"] = {"UKS1": {
    "Age": 13.1, "Age_Err_1os": 0.93, "Age_Err_1us": -1.29,
    "Fe_H": -0.98, "Fe_H_Err": 0.11,
}}

paper_age["Fernandez20"] = 2020

### VVVCL001
Fernandez21: https://ui.adsabs.harvard.edu/abs/2021ApJ...908L..42F/abstract  
High resolution abundance analysis + isochrone fitting  
Alpha Available


In [44]:
individual_data["Fernandez21"] = {"VVVCL001": {
    "Age": 11.9, "Age_Err_1os": 3.12, "Age_Err_1us": -4.05,
    "Fe_H": -2.45, "Fe_H_Err": 0.24,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist": 8.22, "dist_err_1os": 1.84, "dist_err_1us": -1.93
}}

paper_age["Fernandez21"] = 2020

### Djorg2
Ortolani19: https://ui.adsabs.harvard.edu/abs/2019A%26A...627A.145O/abstract   
Isochrone fitting, BaSTi uncorrected?

In [45]:
individual_data["Ortolani19"] = {"Djorg2": {
    "Age": 12.7, "Age_Err_1os": 0.72, "Age_Err_1us": -0.69,
    "Fe_H": -1.11, "Fe_H_Err": 0.03,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist": 8.75, "dist_err": 0.12
}}

paper_age["Ortolani19"] = 2019

### NGC6522
Barbuy21: https://ui.adsabs.harvard.edu/abs/2021A%26A...654A..29B/abstract
Spectra, Flames GIRAFEE  
Alpha available

In [46]:
individual_data["Barbuy21"] = {"NGC6522": {
    "Age": 12.7, "Age_Err_1os": 0.72, "Age_Err_1us": -0.69,
    "Fe_H": -1.05, "Fe_H_Err": 0.20,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist": 8.75, "dist_err": 0.12,
    "vlos": -15.62, "vlos_err": 7.7
}}

paper_age["Barbuy21"] = 2021

### NGC6528
Munoz18: https://ui.adsabs.harvard.edu/abs/2018A%26A...620A..96M/abstract   
Alpha available too.

In [47]:
individual_data["Munoz18"] = {"NGC6528": {
    "Fe_H": -0.14, "Fe_H_Err": 0.03,
}}

paper_age["Munoz18"] = 2018

### NGC6544
Gran21: https://ui.adsabs.harvard.edu/abs/2021MNRAS.504.3494G/abstract

In [48]:
individual_data["Gran21"] = {"NGC6544": {
    "Fe_H": -1.44, "Fe_H_Err": 0.04,
    "Alpha_Fe": 0.2, "Alpha_Fe_Err": 0.04
}}

paper_age["Gran21"] = 2021

### ESO280
Simpson18:  https://ui.adsabs.harvard.edu/abs/2018MNRAS.477.4565S/abstract   
High Res Spectra

In [49]:
individual_data["Simpson18"] = {"ESO-SC06": {
    "Fe_H": -2.48, "Fe_H_Err_1us": -0.11, "Fe_H_Err_1os": 0.06,
    "Mass": 12e3, "Mass_Err": 2e3,
    "Alpha_Fe": 0.2, "Alpha_Fe_Err": 0.04,
    "dist": 15.2, "dist_err": 2.1,
    "vlos": 92.5, "vlos_err_1os": 2.4, "vlos_err_1us": -1.6
}}

paper_age["Simpson18"] = 2018

### NGC6553
Munoz20: https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.3742M/abstract
High Res Spectra

In [50]:
individual_data["Munoz20"] = {"NGC6553": {
    "Fe_H": -0.14, "Fe_H_Err": 0.07,
    "Alpha_Fe": 0.11, "Alpha_Fe_Err": 0.05
}}

paper_age["Munoz20"] = 2020

### NGC6558
Barbuy18: https://ui.adsabs.harvard.edu/abs/2018A%26A...619A.178B/abstract
High Res Spectra  
Alpha Available

In [51]:
individual_data["Barbuy18"] = {"NGC6558": {
    "Fe_H": -1.17, "Fe_H_Err": 0.10,
}}

paper_age["Barbuy18"] = 2018

### Valenti11
Valenti11: https://ui.adsabs.harvard.edu/abs/2011MNRAS.414.2690V/abstract
High res infrared spectra

In [52]:
individual_data["Valenti11"] = {
    "NGC6624": {"Fe_H": -0.69, "Fe_H_Err": 0.02,
                "Alpha_Fe": 0.39, "Alpha_Fe_Err": 0.02,
                "vlos": 51, "vlos_Err": 3
                },
    "NGC6569": {"Fe_H": -0.79, "Fe_H_Err": 0.02,
                "Alpha_Fe": 0.43, "Alpha_Fe_Err": 0.02,
                "vlos": 47, "vlos_Err": 4
                }
}

paper_age["Valenti11"] = 2011

### NGC6569
Saracino19: https://ui.adsabs.harvard.edu/abs/2019ApJ...874...86S/abstract
Age from averaged isochorne from DSED, VR, and BaSTI

In [53]:
individual_data["Saracino19"] = {"NGC6569": {
    "dist": 10.1, "dist_err": 0.2,
    "Age": 12.8, "Age_Err": 1,
    "Fe_H": -1.17, "Fe_H_Err": 0.10,
}}

paper_age["Saracino19"] = 2019

### NGC6642
Balbinot10: https://ui.adsabs.harvard.edu/abs/2010IAUS..266..357B/abstract

In [54]:
individual_data["Balbinot10"] = {"NGC6642": {
    "Age": 10 * 10.14, "Age_Err_1us": ((10**1.19) - (10**1.14)), "Age_Err_1us": ((10**1.14) - (10**1.05)),
    "Fe_H": -1.80, "Fe_H_Err": 0.2,
    "dist": 8.05, "dist_err": 0.66,
}}

paper_age["Balbinot10"] = 2010

### NGC6809
Rain19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.483.1674R/abstract  
Spectra, Chemical 

Diakogiannis14: https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.3172D/abstract  
Dynamical Mass  

In [55]:
individual_data["Rain19"] = {"NGC6809": {
    "Fe_H": -2.01, "Fe_H_Err": 0.02,
    "Alpha_Fe": 0.4, "Alpha_Fe_Err": 0.04,
}}

paper_age["Rain19"] = 2019

individual_data["Diakogiannis14"] = {"NGC6809": {
    "Mass": 6.1e4, "Mass_Err_1os": 0.51e4, "Mass_Err_1us": -0.88e4,
}}

paper_age["Diakogiannis14"] = 2014

### Palomar 11
Lewis06: https://ui.adsabs.harvard.edu/abs/2006AJ....131.2538L/abstract

In [56]:
individual_data["Lewis06"] = {"Pal11": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Lewis06"] = 2006

### Segue3
Ortolani13: https://ui.adsabs.harvard.edu/abs/2013MNRAS.433.1966O/abstract

Hughes17: https://ui.adsabs.harvard.edu/abs/2017AJ....154...57H/abstract  
Dartmouth Geneva Models. Very, very young!

In [57]:
individual_data["Ortolani13"] = {"Segue3": {
    "Age": 3.2,
    "Fe_H": -0.8,
}}

paper_age["Ortolani13"] = 2013

individual_data["Hughes17"] = {"Segue3": {
    "Fe_H": -0.55, "Fe_H_Err_1us": -0.12, "Fe_H_Err_1os": 0.15,
    "Age": 2.6, "Age_Err": 0.4,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Hughes17"] = 2017

### Laevens3
Longeard19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.490.1498L/abstract

In [58]:
individual_data["Longeard19"] = {"Laevens3": {
    "Age": 13.0, "Age_Err": 1.0,
    "dist": 61.4, "dist_err": 1.0,
    "Fe_H": -1.8, "Fe_H_Err": 0.1,
    "vlos": -70.2, "vlos_err": 0.5
}}

paper_age["Longeard19"] = 2019

### Palomar 11
Lewis06: https://ui.adsabs.harvard.edu/abs/2006AJ....131.2538L/abstract

In [59]:
individual_data["Lewis06"] = {"Pal11": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Lewis06"] = 2006

### NGC4833
Carretta14a:  https://ui.adsabs.harvard.edu/abs/2014A%26A...564A..60C/abstract

In [60]:
individual_data["Carretta14a"] = {"NGC4833": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Carretta14a"] = 2014

### NGC6388
Carretta21b: https://ui.adsabs.harvard.edu/abs/2021arXiv211112721C/abstract  
Metallicity only

In [61]:
# individual_data["Carretta21"] = {"NGC6388": {
#     "Fe_H":-0.480,
#     "Fe_H":-0.488,
# }}

# paper_age["Caretta21"] = 2021

### Terzan8
Carretta14b: https://ui.adsabs.harvard.edu/abs/2014A%26A...561A..87C/abstract

In [62]:
individual_data["Carretta14b"] = {"Terzan8":
                                  {"Fe_H": -2.27, "Fe_H_Err": 0.03}
                                  }

paper_age["Carretta14b"] = 2014

### Liller1
Ferraro21: https://ui.adsabs.harvard.edu/abs/2021NatAs...5..311F/abstract  
Multiple populations, we use the oldest. Not clear if it is a genuine GC?

In [63]:
individual_data["Ferraro21"] = {"Liller1":
                                {"Fe_H": -0.4, "Alpha_Fe": 0.2,
                                 "Age": 12, "Age_Err": 1.5,
                                 }
                                }

paper_age["Ferraro21"] = 2021

## Add Indiviudal Data

In [64]:
data = add_individual_data(data, individual_data)

Kerber18
Dias15
New Prop: Mg_Fe
New Prop: Mg_Fe_Err
Vandenberg16
Bonatto20
Weisz16
Hempel14
Kim16
Munoz12
Davoust11
Andreas17
Buckner16
Ryu18
Bragaglia15
Simpson17
Cornish06
Johnson17
Arellano15
Cardelano20
Kerber20
Barbuy16
Romero21
Vilanova19
Barba19
DAntona22
Johnson19
Souza21
Pallanca21
Munoz17
Fernandez20
Fernandez21
Ortolani19
Barbuy21
Munoz18
Gran21
Simpson18
Munoz20
Barbuy18
Valenti11
Saracino19
Balbinot10
Rain19
Diakogiannis14
Lewis06
Ortolani13
Hughes17
Longeard19
Carretta14a
Carretta14b
Ferraro21


# List Data

In [65]:
alt_name_dic = create_alt_name_list(name_parse)

In [66]:
def number_props_data(data, props=[]):
    names = list(data.keys())
    N_total = len(names)
    for p in props:
        print(f"Checking Property {p}")
        missing = []
        for n in names:
            if p not in list(data[n].keys()):
                missing.append(n)
        N_prop = len(missing)
        print(f"{N_total - N_prop} / {N_total}  have GCs have {p}")
        print(f"{N_prop} / {N_total} do not have GCs have {p}")
        print(missing)
        print("\n")
    return
number_props_data(data, props=["vlos", "dist", "Age", "Fe_H", "Mass", "Mv"])

Checking Property vlos
176 / 181  have GCs have vlos
5 / 181 do not have GCs have vlos
['2MASS', 'Ruprecht106', 'Berk29', 'Saurer1', 'Gran4']


Checking Property dist
177 / 181  have GCs have dist
4 / 181 do not have GCs have dist
['2MASS', 'Ruprecht106', 'Berk29', 'Saurer1']


Checking Property Age
129 / 181  have GCs have Age
52 / 181 do not have GCs have Age
['Ko2', 'ESO93', 'Ko1', 'BH140', 'Ryu059', 'NGC6139', 'Terzan3', 'FSR1735', 'NGC6293', 'NGC6316', 'NGC6325', 'NGC6333', 'NGC6355', 'IC1257', 'Terzan2', 'Terzan4', 'NGC6380', 'Terzan1', 'Ton2', 'NGC6401', 'VVVCL002', 'Djorg1', 'Terzan5', 'Terzan6', 'NGC6453', 'Gran1', 'Pfleiderer2', 'Terzan9', 'NGC6517', 'Terzan10', 'NGC6539', 'NGC6540', 'ESO-SC06', 'NGC6558', 'IC1276', 'Terzan12', 'BH261', 'Mercer5', 'Pal8', 'Ryu879', 'NGC6749', 'NGC6760', 'Pal10', 'Pal13', 'GLIMPSE01', 'GLIMPSE02', 'SagittariusII', '2MASS', 'Gran2', 'Gran3', 'Gran4', 'Gran5']


Checking Property Fe_H
169 / 181  have GCs have Fe_H
12 / 181 do not have GCs have F

# Final Data
- Sort by year for now 
- Check errors coherent
- Symmetric errors vs asymmetric

### Improvements?
- If age-metallicity found together, prefer the result
- Average the result of many?
- Consistent isochrone usage?

## Find Best

In [67]:
def find_best_data(data, error_blacklist=[],alts=True):
    '''Error blacklist - ignore errors for a given property'''
    names = list(data.keys())
    best_data = {}
    best_refs = {}
    for gc in names:
        best_data[gc], best_refs[gc] = find_best_individual_data(data[gc], err_blacklist=error_blacklist)
        
    
    if alts:
        alt_names = find_alternative_names(names)
        for alt_n, n in zip(alt_names, names):
            best_data[n]["Name2"] = alt_n
            
    return best_data, best_refs

def find_alternative_names(names):
    alt_names = np.empty_like(names)
    for i,n  in enumerate(names):
        alt_name = alt_name_dic.get(n,["-"])
        N_alts = len(alt_name)
        if N_alts>0:
            alt_name = alt_name[0]
            if alt_name == n:
                if N_alts>1:
                    alt_name = alt_name[1]
                else:
                    alt_name = "-"
        else:
            alt_name = "-"
            alt_names[i] = alt_name
    
    return alt_names


def find_best_individual_data(gc_data, err_blacklist=[]):
    '''Error blacklist - ignore errors for a given property'''
    all_props = list(gc_data.keys())
    props = [p for p in all_props if "err" not in p]
    best_data = {}
    best_refs = {}
    for x in props:
        x_err = x + "_err"
        error_condition = (x not in err_blacklist) * (x_err in all_props)
        if error_condition:
            papers = np.array(list(gc_data[x + "_err"].keys()))
        else:
            papers = np.array(list(gc_data[x].keys()))

        years = np.array([paper_age[p] for p in papers])
        sort_papers = papers[np.argsort(years)[::-1]]
        best_paper = sort_papers[0]
        best_data[x] = gc_data[x][best_paper]
        best_refs[x] = best_paper
        if error_condition:
            best_data[x_err] = gc_data[x_err][best_paper]
            best_refs[x_err] = best_paper
            
    return best_data, best_refs

In [68]:
# best_gc_data, best_refs = find_best_data(data, error_blacklist=["Fe_H"])
best_gc_data, best_refs = find_best_data(data, error_blacklist=[])

In [69]:
best_data = gc_dic_to_data_dic(best_gc_data)

## List Data

In [70]:
sky_obs = ["ra_deg", "dec_deg", "dist", "dist_err", "vlos", "vlos_err", "pmra", "pmra_err", "pmdec_err", "pmcorr"]
AMR = ["Age", "Age_err", "Fe_H", "Fe_H_err"]
Mass = ["Mass"]
Name2 =["Name2"]
basic_props = Name2 + sky_obs + AMR + Mass
print(basic_props)

['Name2', 'ra_deg', 'dec_deg', 'dist', 'dist_err', 'vlos', 'vlos_err', 'pmra', 'pmra_err', 'pmdec_err', 'pmcorr', 'Age', 'Age_err', 'Fe_H', 'Fe_H_err', 'Mass']


## Require Dynamics

In [75]:
def filter_data_dic(data_dic, need_props= sky_obs, included_props = basic_props):
    nGC= len(data_dic["Name"])
    filt = np.ones((nGC), dtype=bool)
    for p in need_props:
        filt = filt* np.isfinite(data_dic[p])
    if included_props is None:
        included_props = list(data_dic.keys())
        
    filt_data_dic = {p: data_dic[p][filt] for p in included_props}
    print(f"Filtered data. {filt.sum()} of {len(filt)} GCs remaining")
    print("Cut GCs:")
    print(data_dic["Name"][np.logical_not(filt)])
        
    return filt_data_dic
sky_data_dic = filter_data_dic(best_data,need_props=sky_obs, included_props=basic_props)

dict_keys(['Name', 'ra_deg', 'dec_deg', 'dist', 'dist_err', 'vlos', 'vlos_err', 'pmra', 'pmra_err', 'pmdec', 'pmdec_err', 'pmcorr', 'l_deg', 'b_deg', 'Mass', 'Mv', 'Age', 'Age_err', 'Fe_H', 'Fe_H_err', 'Alpha_Fe', 'm_Mv', 'm_Mv_err', 'Name2', 'Mass_err', 'Mv_err', 'RGC', 'Alpha_Fe_err', 'Mg_Fe', 'Mg_Fe_Err', 'mua_cos', 'mud', 'mul_cosb', 'mub'])
Filtered data. 173 of 181 GCs remaining
Cut GCs:
['2MASS' 'Ruprecht106' 'Berk29' 'Saurer1' 'Gran2' 'Gran3' 'Gran4' 'Gran5']


In [72]:
print(sky_data_dic.keys())

dict_keys(['Name2', 'ra_deg', 'dec_deg', 'dist', 'dist_err', 'vlos', 'vlos_err', 'pmra', 'pmra_err', 'pmdec_err', 'pmcorr', 'Age', 'Age_err', 'Fe_H', 'Fe_H_err', 'Mass'])


## Require Dynamics, AMR, Mass/Mv